In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, minmax_scale
from sklearn.metrics import log_loss
from scipy.optimize import minimize
import pandas as pd
#from scipy.special import expit
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
import math

from itertools import product
from sys import stdout as out

# from interpret.glassbox import ExplainableBoostingClassifier
# from interpret import show


%matplotlib inline


In [76]:

from print_versions import print_versions

In [74]:
print_versions(globals())

pandas==2.2.2
scipy==1.14.1
numpy==1.26.4
matplotlib==3.9.2
ipykernel==6.29.5


In [75]:
import sys
print("Python version:", sys.version)

Python version: 3.12.5 | packaged by Anaconda, Inc. | (main, Sep 12 2024, 18:18:29) [MSC v.1929 64 bit (AMD64)]


In [2]:
dataPath="C:\\Users\\xxxxxxxx\\xxxxx\\LR_Rudin_Tests\\data\\"

In [3]:
Training= pd.read_csv(dataPath+'2025-01-15 mammographicmass training set.csv')
Test = pd.read_csv(dataPath+'2025-01-15 mammographicmass test set.csv')

In [4]:
Training.head()

,row_number,Dependent,BI-RADS,Age,Shape,Margin,Density
0,1,0,4,46,1,1,1
1,3,0,4,78,1,1,1
2,4,0,4,49,2,1,1
3,5,0,4,44,4,1,1
4,6,0,5,67,2,4,1


In [5]:
Test.head()

,row_number,Dependent,BI-RADS,Age,Shape,Margin,Density
0,2,0,4,70,1,1,1
1,7,0,4,35,1,1,2
2,11,0,4,53,1,1,2
3,12,0,2,76,1,1,2
4,20,0,4,44,2,1,2


In [6]:
column_names = Training.columns.tolist()

print(column_names)

['row_number', 'Dependent', 'BI-RADS', 'Age ', 'Shape', 'Margin', 'Density']


In [11]:
#randomDF = pd.read_csv(dataPath+'jsmRandoms.csv')
posData = Training[Training["Dependent"]==1].copy().iloc[:,1:]
posData = posData.iloc[:,1:]     #the prior line already took off one column
negData = Training[Training["Dependent"]==0].copy().iloc[:,1:]
negData = negData.iloc[:,1:]

In [12]:
posData

,BI-RADS,Age,Shape,Margin,Density
311,4,42,1,1,1
312,4,74,1,1,1
313,5,40,4,1,1
314,5,56,4,3,1
315,4,70,4,2,2
...,...,...,...,...,...
610,5,87,4,5,3
611,5,87,4,5,3
612,4,57,4,4,4
613,5,57,4,4,4


In [13]:
#X1 = Training.iloc[:,1:]
X1 = Training.iloc[:,2:]
y1 = Training["Dependent"]

In [14]:
#X2 = Test.iloc[:,1:]
X2 = Test.iloc[:,2:]
y2 = Test["Dependent"]

In [15]:
X1

,BI-RADS,Age,Shape,Margin,Density
0,4,46,1,1,1
1,4,78,1,1,1
2,4,49,2,1,1
3,4,44,4,1,1
4,5,67,2,4,1
...,...,...,...,...,...
610,5,87,4,5,3
611,5,87,4,5,3
612,4,57,4,4,4
613,5,57,4,4,4


In [16]:
X2

,BI-RADS,Age,Shape,Margin,Density
0,4,70,1,1,1
1,4,35,1,1,2
2,4,53,1,1,2
3,2,76,1,1,2
4,4,44,2,1,2
...,...,...,...,...,...
210,5,76,4,5,3
211,5,80,4,5,3
212,5,84,4,5,3
213,5,87,4,5,3


In [17]:
y1.head()

0    0
1    0
2    0
3    0
4    0
Name: Dependent, dtype: int64

In [18]:
posData

,BI-RADS,Age,Shape,Margin,Density
311,4,42,1,1,1
312,4,74,1,1,1
313,5,40,4,1,1
314,5,56,4,3,1
315,4,70,4,2,2
...,...,...,...,...,...
610,5,87,4,5,3
611,5,87,4,5,3
612,4,57,4,4,4
613,5,57,4,4,4


In [19]:
total_y1 =  y1.sum()
numCases = len(y1.index)

In [20]:
print(y1.sum(),len(y1.index))

304 615


In [21]:
print(y2.sum(),len(y2.index))

99 215


# Utilities related to calculating and printing lift #s

In [22]:
def LogRegCoeffs(X1,y1):
    #log_reg = LogisticRegression( fit_intercept=True,solver='liblinear', max_iter = 10000)
    log_reg = LogisticRegression(fit_intercept=True, penalty = None, max_iter = 100000, tol=0.00000001)
    #log_reg = LogisticRegression(C=1e42 ,fit_intercept=True, class_weight = 'balanced', solver='liblinear', max_iter = 10000)
    log_reg.fit(X1, y1)
    return np.asarray(log_reg.coef_[0].tolist()), log_reg

In [23]:
# log_reg2 = LogisticRegression(C=1e42 ,fit_intercept=True, class_weight = 'balanced', solver='saga', max_iter = 10000)
# log_reg2.fit(X1, y1)

In [24]:
def calcCumGains(modelX, df, X, sumY,coeffs):
    
    #takes in a model, a complete df, the X data, the sum of the Y's and a set of coeffs
    modelX.coef_ = coeffs     #sticks the coeffs into the model

    myVals = modelX.predict(X)
    #print(myVals.size)
    df["calc1"]=pd.Series(myVals)         # puts the dot product [dot(coefs, Xdata)] inot new column calc1
    df = df.sort_values(by=['calc1'],ascending=False)  #  sorts the dataframe on the calc1
    df['cumGain'] = df['Dependent'].cumsum()/sumY       #creates new 'cumGain' column in dataframe
    return df
    

In [25]:
def PTcalcCumGains(dfz, df, sumY):
    
    #takes in a model, a complete df, the X data, the sum of the Y's and a set of coeffs
    #modelX.coef_ = coeffs     #sticks the coeffs into the model

    #myVals = modelX.predict(X)
    #print(myVals.size)
    #df["calc1"]=dfz['prob']
    df["calc1"]=dfz['prob']         
    df = df.sort_values(by=['calc1'],ascending=False)  #  sorts the dataframe on the calc1
    df['cumGain'] = df['Dependent'].cumsum()/sumY       #creates new 'cumGain' column in dataframe
    return df

In [26]:
#def createFullTable(df):
def createFullTable(df,total_y1,numCases):
    lift = []
    perflift = []
    order = []
    #expProb = y1.sum()/len(y1.index)
    expProb = total_y1/numCases
    CumGains = list(df['cumGain'])

    for num, CG in enumerate(CumGains, start=1):
        lift.append(CG/(num/numCases)) 
        perflift.append(min(total_y1,num)/(num*expProb))
        order.append(num)

    df["Lift"]= lift
    df["perfLift"]= perflift
    df["Rank"]= order

In [27]:
def LiftTable(df):
    Lift = list(df['Lift'])
    numCases = len(Lift)
    checkpoints = [ 0.01, 0.05, 0.10, 0.20, 0.25, 0.30, 0.40, 0.50, 0.60, 1.0]
    for point in checkpoints:
        print("For point {} Lift = {}".format(point, Lift[int(numCases*point)-1]))
    

In [28]:
def LiftTable3(rSeed,epoch,df1, df2, df3):
    Lift1 = list(df1['Lift'])
    Lift2 = list(df2['Lift'])
    Lift3 = list(df3['Lift'])
    
    numCases1 = len(Lift1)
    numCases2 = len(Lift2)
    numCases3 = len(Lift3)
    checkpoints = [ 0.05, 0.10, 0.20, 0.25, 0.30 , 0.50, 0.75, 1.0]
    for point in checkpoints:
        #print("For point  {:.1%} \t Train = {:0.3f} \t  Val= {:0.3f} \tTest= {:0.3f}".format(point, Lift1[int(numCases1*point)-1], Lift2[int(numCases2*point)-1], Lift3[int(numCases3*point)-1]))
        mylist = [rSeed,epoch,point, Lift1[int(numCases1*point)-1], Lift2[int(numCases2*point)-1], Lift3[int(numCases3*point)-1]] 
        #df.loc[len(df)] = list
        dfRecord.loc[len(dfRecord)] = mylist

In [29]:
def Liftsummary(df):
    Lift = list(df['Lift'])
    numCases = len(Lift)
    checkpoints = [ 0.01, 0.05, 0.10, 0.25, 0.30, 0.50, 1.0]
    for point in checkpoints:
        #count = 6
        count = df.head(int(numCases * point))['Dependent'].sum()
        print("For point {:.1%} Lift = {:0.3f}    Count = {} ".format(point, Lift[int(numCases*point)-1],count ))
                                                         

# Model Building

In [30]:
lin_reg = LinearRegression(fit_intercept=False)    # Note - this model is not atually used. The object itself has methods useful
lin_reg.fit(X1, y1)                                # to do some calcultion

LinearRegression(fit_intercept=False)

In [31]:
log_reg_coeffs, log_reg = LogRegCoeffs(X1,y1)

In [32]:
log_reg

LogisticRegression(max_iter=100000, penalty=None, tol=1e-08)

In [33]:
#################################################################################################

In [34]:
feature_coeff = pd.DataFrame(np.atleast_2d(log_reg_coeffs), columns= X1.columns )
#feature_coeff = pd.DataFrame(np.atleast_2d(log_reg_coeffs))
#feature_coeff.to_csv(dataPath + "LogReg_Mammog.csv")

In [35]:
log_reg_coeffs

array([ 1.52583355,  0.05323493,  0.44003223,  0.31074736, -0.13328209])

In [36]:
feature_coeff.T

,0
BI-RADS,1.525834
Age,0.053235
Shape,0.440032
Margin,0.310747
Density,-0.133282


In [37]:
#feature_coeff.to_csv(dataPath +"LogRegCoefs.csv")

In [38]:
LRTrainTable = calcCumGains(lin_reg,Training,X1,y1.sum(),log_reg_coeffs)  #Run1Table
createFullTable(LRTrainTable,y1.sum(),len(y1.index))
LRTrainTable

,row_number,Dependent,BI-RADS,Age,Shape,Margin,Density,calc1,cumGain,Lift,perfLift,Rank
607,820,1,6,80,4,5,3,16.327815,0.003289,2.023026,2.023026,1
512,691,1,6,71,4,4,3,15.537954,0.006579,2.023026,2.023026,2
411,550,1,6,72,4,3,3,15.280441,0.009868,2.023026,2.023026,3
611,825,1,5,87,4,5,3,15.174626,0.013158,2.023026,2.023026,4
610,824,1,5,87,4,5,3,15.174626,0.016447,2.023026,2.023026,5
...,...,...,...,...,...,...,...,...,...,...,...,...
409,548,1,0,72,4,3,3,6.125440,1.000000,1.006547,1.006547,611
34,50,0,3,21,1,1,3,6.046368,1.000000,1.004902,1.004902,612
11,16,0,2,35,2,1,2,5.839137,1.000000,1.003263,1.003263,613
68,91,0,2,40,1,1,3,5.531998,1.000000,1.001629,1.001629,614


In [39]:
LRTestTable = calcCumGains(lin_reg,Test,X2,y2.sum(),log_reg_coeffs)   #Run2Table
createFullTable(LRTestTable,y2.sum(),len(y2.index))
LRTestTable

,row_number,Dependent,BI-RADS,Age,Shape,Margin,Density,calc1,cumGain,Lift,perfLift,Rank
213,826,1,5,87,4,5,3,15.174626,0.010101,2.171717,2.171717,1
212,822,1,5,84,4,5,3,15.014922,0.020202,2.171717,2.171717,2
115,423,0,5,84,4,5,3,15.014922,0.020202,1.447811,2.171717,3
211,819,1,5,80,4,5,3,14.801982,0.030303,1.628788,2.171717,4
183,720,1,5,85,4,4,3,14.757409,0.040404,1.737374,2.171717,5
...,...,...,...,...,...,...,...,...,...,...,...,...
15,51,0,4,21,1,1,3,7.572201,0.989899,1.008665,1.018957,211
177,686,1,0,71,4,4,3,6.382952,1.000000,1.014151,1.014151,212
17,55,0,3,23,1,1,3,6.152837,1.000000,1.009390,1.009390,213
103,390,0,0,58,4,4,3,5.690898,1.000000,1.004673,1.004673,214


# Rudin method

In [42]:
posArray = posData.values
negArray = negData.values


In [43]:
def my_vectorized_exp(m):
    
    #m = np.float128(m) 

    return np.exp(-m)

In [46]:
coef = log_reg_coeffs

In [47]:
#posArray

In [48]:
#coef

In [49]:
posScores = np.matmul(posArray, coef)

# Set p

In [50]:
p=5

In [51]:
#posArray

In [52]:
def myFunc(coef):
    posScores = np.matmul(posArray, coef)
    negScores = np.matmul(negArray, coef)
    posTile = np.tile(posScores, (np.shape(negScores)[0], 1))  
    negTile = np.tile(np.atleast_2d(negScores).transpose(), (1, np.shape(posScores)[0]))
    target = np.power(my_vectorized_exp(posTile - negTile).sum(axis=1),p).sum()
    #target: float64 = np.power(my_vectorized_exp(posTile - negTile).sum(axis=1),p).sum()
    #print(target)
    return target

In [53]:
myFunc(log_reg_coeffs)

2.745018150598633e+21

In [54]:
myFunc(log_reg_coeffs)

2.745018150598633e+21

In [55]:
feature_coeff = pd.DataFrame(columns = X1.columns )
feature_coeff2 = pd.DataFrame(columns = X1.columns )
#feature_coeff = pd.DataFrame()
feature_coeff                 #Aen empty dataframe with the columns I want

,BI-RADS,Age,Shape,Margin,Density


In [56]:

res = minimize(myFunc, log_reg_coeffs, method='Nelder-Mead', options={'maxiter':100000})

feature_coeff.loc[len(feature_coeff.index)] = res.x

In [57]:
res

       message: Maximum number of iterations has been exceeded.
       success: False
        status: 2
           fun: 215905104434061.12
             x: [ 2.717e-01  8.902e-03  7.692e-02  4.436e-02 -4.515e-02]
           nit: 100000
          nfev: 697634
 final_simplex: (array([[ 2.717e-01,  8.902e-03, ...,  4.436e-02,
                        -4.515e-02],
                       [ 2.717e-01,  8.902e-03, ...,  4.436e-02,
                        -4.515e-02],
                       ...,
                       [ 2.717e-01,  8.902e-03, ...,  4.436e-02,
                        -4.515e-02],
                       [ 2.717e-01,  8.902e-03, ...,  4.436e-02,
                        -4.515e-02]]), array([ 2.159e+14,  2.159e+14,  2.159e+14,  2.159e+14,
                        2.159e+14,  2.159e+14]))

In [58]:
res.x

array([ 0.2717422 ,  0.00890222,  0.07692203,  0.04435568, -0.04515344])

In [59]:
# #array([-3.81539912e-06, -2.41570774e-02,  5.28935021e-01, -4.63182842e-04,
#        -5.38534544e-05, -6.88916060e-03,  3.22992283e-01,  1.55454081e-01,
#        -8.23957812e-01,  9.32114821e-02])

In [60]:
feature_coeff

,BI-RADS,Age,Shape,Margin,Density
0,0.271742,0.008902,0.076922,0.044356,-0.045153


In [61]:
########################################################################

In [62]:
#feature_coeff.to_csv(dataPath + "RudinMamm_p5.csv")

In [63]:
feature_coeff.T

,0
BI-RADS,0.271742
Age,0.008902
Shape,0.076922
Margin,0.044356
Density,-0.045153


In [64]:
RudinTrainTable = calcCumGains(lin_reg,Training,X1,total_y1,res.x)    #PowellTrainTable
createFullTable(RudinTrainTable,total_y1,numCases)
RudinTrainTable.head()

,row_number,Dependent,BI-RADS,Age,Shape,Margin,Density,calc1,cumGain,Lift,perfLift,Rank
607,820,1,6,80,4,5,3,2.736637,0.003289,2.023026,2.023026,1
512,691,1,6,71,4,4,3,2.612161,0.006579,2.023026,2.023026,2
411,550,1,6,72,4,3,3,2.576708,0.009868,2.023026,2.023026,3
405,543,1,6,68,4,3,3,2.541099,0.013158,2.023026,2.023026,4
611,825,1,5,87,4,5,3,2.527210,0.016447,2.023026,2.023026,5


In [65]:
RudinTestTable = calcCumGains(lin_reg,Test,X2,y2.sum(),res.x) 
createFullTable(RudinTestTable,y2.sum(),len(y2.index))
RudinTestTable.head()
#y1.sum(),len(y1.index)

,row_number,Dependent,BI-RADS,Age,Shape,Margin,Density,calc1,cumGain,Lift,perfLift,Rank
213,826,1,5,87,4,5,3,2.527210,0.010101,2.171717,2.171717,1
212,822,1,5,84,4,5,3,2.500504,0.020202,2.171717,2.171717,2
115,423,0,5,84,4,5,3,2.500504,0.020202,1.447811,2.171717,3
183,720,1,5,85,4,4,3,2.465050,0.030303,1.628788,2.171717,4
211,819,1,5,80,4,5,3,2.464895,0.040404,1.737374,2.171717,5


In [66]:
# RudinTestTable.to_csv(dataPath+"rudenTest1p.csv")
# LRTestTable.to_csv(dataPath + "LRTest.csv")

In [67]:
def Liftsummary(df):
    Lift = list(df['Lift'])
    numCases = len(Lift)
    checkpoints = [ 0.01, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.50, 1.0]
    for point in checkpoints:
        #count = 6
        #count = df.head(int(numCases * point))['Dependent'].sum()
        count = df.head(math.ceil(numCases * point))['Dependent'].sum()
        #print("For point {:.1%} Lift = {:0.3f}    Count = {} ".format(point, Lift[int(numCases*point)-1],count ))
        print("For point {:.1%} Lift = {:0.3f}    Count = {} ".format(point, Lift[round(numCases*point)-1],count ))
                                                                   

# Log Reg Training and Test

In [68]:
Liftsummary(LRTrainTable)

For point 1.0% Lift = 2.023    Count = 7 
For point 5.0% Lift = 1.958    Count = 30 
For point 10.0% Lift = 1.958    Count = 60 
For point 15.0% Lift = 1.935    Count = 89 
For point 20.0% Lift = 1.924    Count = 117 
For point 25.0% Lift = 1.905    Count = 145 
For point 30.0% Lift = 1.880    Count = 172 
For point 50.0% Lift = 1.681    Count = 256 
For point 100.0% Lift = 1.000    Count = 304 


In [69]:
Liftsummary(LRTestTable)

For point 1.0% Lift = 2.172    Count = 2 
For point 5.0% Lift = 1.974    Count = 10 
For point 10.0% Lift = 2.073    Count = 21 
For point 15.0% Lift = 2.104    Count = 32 
For point 20.0% Lift = 2.020    Count = 40 
For point 25.0% Lift = 1.930    Count = 48 
For point 30.0% Lift = 1.968    Count = 58 
For point 50.0% Lift = 1.669    Count = 83 
For point 100.0% Lift = 1.000    Count = 99 


# Rudin p-Norm Training and Test P=5

In [70]:
Liftsummary(RudinTrainTable)

For point 1.0% Lift = 2.023    Count = 7 
For point 5.0% Lift = 1.958    Count = 30 
For point 10.0% Lift = 1.958    Count = 60 
For point 15.0% Lift = 1.935    Count = 89 
For point 20.0% Lift = 1.924    Count = 117 
For point 25.0% Lift = 1.905    Count = 145 
For point 30.0% Lift = 1.880    Count = 172 
For point 50.0% Lift = 1.688    Count = 257 
For point 100.0% Lift = 1.000    Count = 304 


In [71]:
Liftsummary(RudinTestTable)

For point 1.0% Lift = 2.172    Count = 2 
For point 5.0% Lift = 1.974    Count = 10 
For point 10.0% Lift = 2.073    Count = 21 
For point 15.0% Lift = 2.104    Count = 32 
For point 20.0% Lift = 2.020    Count = 40 
For point 25.0% Lift = 1.930    Count = 48 
For point 30.0% Lift = 1.934    Count = 58 
For point 50.0% Lift = 1.689    Count = 84 
For point 100.0% Lift = 1.000    Count = 99 


In [72]:
#LRTrainTable.to_csv(dataPath+"LRTrain2.csv")